In [5]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import os
import requests
import shutil
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?voiture ?label ?image WHERE {
  VALUES ?type {
    wd:Q274586   # Voitures de sport
    wd:Q185721   # Roadster
    wd:Q34427    # Voiture de collection
    wd:Q3231690  # Pony car (muscle cars légères)
    wd:Q363288   # Kei car (souvent des JDM)
  }
  
  ?voiture wdt:P31 ?type;    # La voiture doit être d'un type défini ci-dessus
           wdt:P18 ?image;   # Image associée
           rdfs:label ?label. # Nom de la voiture

  FILTER(LANG(?label) IN ("fr", "en")).  # Garde les labels en français ou anglais
}
LIMIT 300
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

# Fonction pour télécharger une image
def download_image(url, folder="images"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_path = os.path.join(folder, os.path.basename(url))
        with open(image_path, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        return image_path
    return None

# Récupérer les résultats de Wikidata
results = get_results(endpoint_url, query)

# Créer un DataFrame avec les résultats
data = []
for result in results["results"]["bindings"]:
    voiture = result["voiture"]["value"]
    label = result["label"]["value"]
    image_url = result["image"]["value"]
    data.append([label, voiture, image_url])

dataframe = pd.DataFrame(data, columns=["label", "voiture", "image"])

# Télécharger les images
dataframe["image_path"] = dataframe["image"].apply(download_image)

# Afficher le DataFrame avec les chemins d'images
print(dataframe)


                        label                                 voiture  \
0              Opel 2.0 litre  http://www.wikidata.org/entity/Q277131   
1                      Opel 6  http://www.wikidata.org/entity/Q277131   
2                Audi Type SS  http://www.wikidata.org/entity/Q194125   
3                Audi Zwickau  http://www.wikidata.org/entity/Q194125   
4                 Audi Type R  http://www.wikidata.org/entity/Q193967   
..                        ...                                     ...   
295            Opel 1.8 litre  http://www.wikidata.org/entity/Q213239   
296               Peugeot 107  http://www.wikidata.org/entity/Q214229   
297            Tesla Roadster  http://www.wikidata.org/entity/Q210893   
298               Peugeot 107  http://www.wikidata.org/entity/Q214229   
299  Ford Explorer Sport Trac  http://www.wikidata.org/entity/Q287647   

                                                 image  \
0    http://commons.wikimedia.org/wiki/Special:File...   
1    ht

In [8]:
import json
from PIL import Image
from PIL.ExifTags import TAGS

metadata = {}

# Fonction pour extraire les métadonnées Exif
def get_exif(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        exif_info = {}

        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                exif_info[tag_name] = value

        # Déterminer l'orientation
        width, height = img.size
        if width > height:
            orientation = "Paysage"
        elif width < height:
            orientation = "Portrait"
        else:
            orientation = "Carré"

        # Stocker les métadonnées
        return {
            "format": img.format,
            "taille": f"{width}x{height}",
            "orientation": orientation,
            "date_creation": exif_info.get("DateTimeOriginal", "Inconnue"),
            "appareil_photo": exif_info.get("Model", "Inconnu"),
        }
    except Exception as e:
        print(f"Erreur avec {image_path} : {e}")
        return None

# Extraire les métadonnées pour chaque image téléchargée
for _, row in dataframe.iterrows():
    image_path = row["image_path"]
    if image_path:
        metadata[row["label"]] = get_exif(image_path)

# Sauvegarder les métadonnées en JSON
with open("metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)

print("Extraction des métadonnées terminée !")

Extraction des métadonnées terminée !
